In [52]:
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
from pathlib import Path
import os

In [ ]:

load_dotenv(Path.home() / "finreport" / ".env")

PG_DSN = os.getenv("PG_DSN", "postgresql:///finreport")  # PostgreSQL (PG) Data Source Name (DSN)
engine = create_engine(PG_DSN)


In [34]:
print(PG_DSN)
print(engine)

postgresql:///finreport
Engine(postgresql:///finreport)


In [54]:
with engine.connect() as conn:
    schema_df = pd.read_sql(
        text("""
            SELECT column_name, data_type, is_nullable
            FROM information_schema.columns
            WHERE table_name = 'news_raw'
            ORDER BY ordinal_position;
        """),
        conn
    )

schema_df


,column_name,data_type,is_nullable
0,id,text,NO
1,published_utc,timestamp with time zone,NO
2,title,text,YES
3,url,text,YES
4,description,text,YES
5,tickers,ARRAY,YES
6,tags,ARRAY,YES
7,published_date_utc,date,YES
8,full_body,text,YES
9,full_body_chars,integer,YES


In [64]:
# --- 2) Show row count ---
with engine.connect() as conn:
    count_df = pd.read_sql(
        text("SELECT COUNT(*) AS total_rows FROM news_raw;"),
        conn
    )

print("🧮 Total number of rows:")
display(count_df)


🧮 Total number of rows:


,total_rows
0,170


In [65]:
# --- 3) Display first 5 rows ---
with engine.connect() as conn:
    preview_df = pd.read_sql(
        text("""
            SELECT *
            FROM news_raw
            ORDER BY published_utc DESC;
        """),
        conn
    )



In [45]:
preview_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   id                  22 non-null     object             
 1   published_utc       22 non-null     datetime64[ns, UTC]
 2   title               22 non-null     object             
 3   url                 22 non-null     object             
 4   description         22 non-null     object             
 5   tickers             22 non-null     object             
 6   tags                22 non-null     object             
 7   published_date_utc  22 non-null     object             
 8   full_body           22 non-null     object             
 9   full_body_chars     22 non-null     int64              
 10  fetched_at          22 non-null     datetime64[ns, UTC]
 11  fetch_status        22 non-null     object             
 12  body_extractor      22 non-null     ob

In [66]:
preview_df['date'] = preview_df['published_utc'].dt.date

# Create boolean columns for each ticker
preview_df['has_nvda'] = preview_df['tickers'].apply(lambda x: 'NVDA' in x)
preview_df['has_tsla'] = preview_df['tickers'].apply(lambda x: 'TSLA' in x)

# Group by date and sum the boolean columns
result = preview_df.groupby('date').agg({
    'has_nvda': 'sum',
    'has_tsla': 'sum'
}).rename(columns={'has_nvda': 'NVDA', 'has_tsla': 'TSLA'}).reset_index()

print(result)

         date  NVDA  TSLA
0  2025-10-01     2    16
1  2025-10-02     5    26
2  2025-10-03     3    20
3  2025-10-04     2     8
4  2025-10-05     2     4
5  2025-10-06     8    17
6  2025-10-07     5    30
7  2025-10-08    10    12


In [48]:
# Check what the tickers column actually looks like
print("Sample tickers values:")
print(preview_df['tickers'].head(10))
print("\nUnique tickers:")
print(preview_df['tickers'].unique())

Sample tickers values:
0    [NVDA]
1    [NVDA]
2    [NVDA]
3    [NVDA]
4    [NVDA]
5    [NVDA]
6    [NVDA]
7    [NVDA]
8    [NVDA]
9    [NVDA]
Name: tickers, dtype: object

Unique tickers:


TypeError: unhashable type: 'list'